In [ ]:
import requests
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout,LayerNormalization
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import re

In [ ]:
url = "https://www.gutenberg.org/files/100/100-0.txt"
response = requests.get(url)
text_data = response.text[:500000]

with open("shakespeare.txt", "w", encoding="utf-8") as file:
    file.write(text_data)

In [ ]:

with open("shakespeare.txt", "r", encoding="utf-8") as file:
    corpus = file.read().lower()

corpus = re.sub(r'[^a-zA-Z.!?\s]', '', corpus)
corpus = re.sub(r'\s+', ' ', corpus).strip()

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([corpus])
sequence = tokenizer.texts_to_sequences([corpus])[0]

vocab_size = len(tokenizer.word_index) + 1
print(f"Vocabulary Size: {vocab_size}")

Vocabulary Size: 8221


In [ ]:
seq_length = 40
input_sequences = [sequence[i - seq_length:i + 1] for i in range(seq_length, len(sequence))]
input_sequences = np.array(input_sequences)
X=input_sequences[:, :-1]
X = np.array(X).reshape(-1, seq_length)
y=input_sequences[:, -1]
y = to_categorical(y, num_classes=vocab_size)

In [ ]:
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=256, input_length=seq_length),
    Bidirectional(LSTM(256, return_sequences=True)),
    Dropout(0.2),
    LayerNormalization(),
    Bidirectional(LSTM(256)),
    Dropout(0.2),
    LayerNormalization(),
    Dense(256, activation=tf.keras.activations.swish),
    Dropout(0.2),
    Dense(vocab_size, activation="softmax")
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
optimizer = Adam(learning_rate=0.001, decay=1e-6)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ layer_normalization                  │ ?                           │     0 (unbuilt) │
│ (LayerNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ layer_normalization_1                │ ?                           │     0 (unbuilt) │
│ (LayerNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train the model with early stopping
early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)
model.fit(X, y, epochs=65, batch_size=32, callbacks=[early_stopping])

Epoch 1/65
2706/2706 ━━━━━━━━━━━━━━━━━━━━ 76s 23ms/step - loss: 6.9726
Epoch 2/65
2706/2706 ━━━━━━━━━━━━━━━━━━━━ 60s 22ms/step - loss: 6.5158
Epoch 3/65
2706/2706 ━━━━━━━━━━━━━━━━━━━━ 59s 22ms/step - loss: 6.2950
Epoch 4/65
2706/2706 ━━━━━━━━━━━━━━━━━━━━ 82s 22ms/step - loss: 6.1065
Epoch 5/65
2706/2706 ━━━━━━━━━━━━━━━━━━━━ 81s 22ms/step - loss: 5.9333
Epoch 6/65
2706/2706 ━━━━━━━━━━━━━━━━━━━━ 58s 22ms/step - loss: 5.7893
Epoch 7/65
2706/2706 ━━━━━━━━━━━━━━━━━━━━ 83s 22ms/step - loss: 5.6591
Epoch 8/65
2706/2706 ━━━━━━━━━━━━━━━━━━━━ 81s 22ms/step - loss: 5.5191
Epoch 9/65
2706/2706 ━━━━━━━━━━━━━━━━━━━━ 82s 22ms/step - loss: 5.4107
Epoch 10/65
2706/2706 ━━━━━━━━━━━━━━━━━━━━ 82s 22ms/step - loss: 5.2995
Epoch 11/65
2706/2706 ━━━━━━━━━━━━━━━━━━━━ 82s 22ms/step - loss: 5.1894
Epoch 12/65
2706/2706 ━━━━━━━━━━━━━━━━━━━━ 59s 22ms/step - loss: 5.1015
Epoch 13/65
2706/2706 ━━━━━━━━━━━━━━━━━━━━ 59s 22ms/step - loss: 5.0048
Epoch 14/65
2706/2706 ━━━━━━━━━━━━━━━━━━━━ 82s 22ms/step - loss: 4.9170
E

In [ ]:
def generate_text(seed_text, next_words, model, tokenizer, seq_length, temperature=0.8):
    result = seed_text
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([result])[0]
        token_list = pad_sequences([token_list], maxlen=seq_length, padding='pre')
        predictions = model.predict(token_list, verbose=0)[0]
        predictions = np.log(predictions + 1e-8) / temperature
        exp_preds = np.exp(predictions)
        predictions = exp_preds / np.sum(exp_preds)
        next_word_index = np.random.choice(range(len(predictions)), p=predictions)
        next_word = tokenizer.index_word.get(next_word_index, "")
        if next_word == "":
            continue
        result += " " + next_word
        if next_word in [".", "!", "?"]:
            result += "\n"
    return result

In [ ]:
seed_text = "Once upon a time in a distant land"
generated_text = generate_text(seed_text, next_words=500, model=model, tokenizer=tokenizer, seq_length=seq_length, temperature=0.7)
print(generated_text)

Once upon a time in a distant land sheeps death and therefore be hired i did not see my wife celia is it speak no more but indeed he has sent them for mine own caesar let me see aside cleopatra no man wheres your whipping and thank me not that i am so threatningly replies love did with me meet in much and to be gone my father makes well till i will be denied a man amiens why then the queen might have spoke already with it parolles i am a knave but an inch of time will you not see your departure hence it must not be our report diana to antony give me leave you say sir to have he was whipped for good lady i can not say sir we are not given him to have his accusations agrippa who has a proper fool the gods determine this antony i faint why celia is it clown o madam hes well helena marry sir king what is he parolles so come sir enobarbus i thank you iras i hear you are antony orlando i come but here to entreat you to know yourself rosalind come peace i pray you mar no more more than told f

In [ ]:
model.save("lstm_model.keras")

In [ ]:
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer

with open("shakespeare.txt", "r", encoding="utf-8") as file:
    shakespeare_text = file.read()

tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts([shakespeare_text])

# Step 3: Save Tokenizer
with open("tokenizer1.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

print("Tokenizer trained and saved successfully!")


Tokenizer trained and saved successfully!


In [ ]:
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

lstm_model = load_model("lstm_model.h5", compile=False)

with open("tokenizer1.pkl", "rb") as f:
    tokenizer = pickle.load(f)

def generate_lstm_text(seed_text, next_words=50, max_sequence_length=10, temperature=0.8):
    """Generates text using the trained LSTM model with temperature scaling."""
    for _ in range(next_words):
        input_seq = tokenizer.texts_to_sequences([seed_text])
        input_seq = pad_sequences(input_seq, maxlen=max_sequence_length, padding="pre")

        predictions = lstm_model.predict(input_seq, verbose=0)[0]

        predictions = np.log(predictions + 1e-8) / temperature
        exp_preds = np.exp(predictions)
        probabilities = exp_preds / np.sum(exp_preds)

        predicted_word_index = np.random.choice(len(probabilities), p=probabilities)

        predicted_word = tokenizer.index_word.get(predicted_word_index, None)

        if predicted_word:
            seed_text += " " + predicted_word
        else:
            print("Prediction resulted in an unknown word. Stopping generation.")
            break

    return seed_text

print(generate_lstm_text("once upon a time", next_words=100, max_sequence_length=20, temperature=0.6))
print("\nGenerated Text:\n", generated_text)


once upon a time daughter not when i recompense a noble else and past him might trust come dear i know not night house do good of this young unpolicied mind displeasure it is nor as were caesar is to be fellows success can i have hold ” the wilt is very more tie you love dead his more bertram once morgan florence orlando marry to i’ll i’ lack spirit kiss never iii should now ’tis like antony what hard thou give fill with this beloved first an he use not time come dear i forth to say that he use her other bravely

Generated Text:
 once upon a time fortune would i’ll but dercetus arden an shall be chin take dissuade the sister part of the living gentle needless stand scene knows forest a ever in the lepidus scene knows home a ever in the count lepidus scene pardon farewell another ever in vi lepidus scene knows farewell another


In [ ]:
import sys

print(sys.version)

3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]


In [ ]:
pip install flask_cors

#GPT2

In [ ]:
import wandb
wandb.init(project="your_project_name", anonymous="allow")


<IPython.core.display.Javascript object>

wandb: (1) Private W&B dashboard, no account required
wandb: (2) Use an existing W&B account


wandb: Enter your choice: 1


wandb: You chose 'Private W&B dashboard, no account required'
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: anony-mouse-292019096749499281 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which

In [ ]:
from datasets import load_dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling

dataset = load_dataset("wikitext", "wikitext-103-raw-v1")

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

def tokenize_and_create_labels(examples):
    input_ids = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512).input_ids
    labels = input_ids.copy()
    return {"input_ids": input_ids, "labels": labels}

tokenized_dataset = dataset.map(tokenize_and_create_labels, batched=True)

train_dataset_size = min(10000, len(tokenized_dataset["train"]))
eval_dataset_size = min(100, len(tokenized_dataset["validation"]))

train_dataset = tokenized_dataset["train"].shuffle(seed=42).select(range(train_dataset_size))
eval_dataset = tokenized_dataset["validation"].shuffle(seed=42).select(range(eval_dataset_size))

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

training_args = TrainingArguments(
    output_dir="./gpt2_finetuned",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir="./logs",
    save_steps=500,
    save_total_limit=2,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)


trainer.train()

trainer.save_model("./gpt2_finetuned_model")


Map:   0%|          | 0/1801350 [00:00<?, ? examples/s]

Map:   0%|          | 0/3760 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,3.304900,3.420286


In [ ]:
trainer.save_model("./gpt2_finetuned_model")
tokenizer.save_pretrained("./gpt2_finetuned_model")


('./gpt2_finetuned_model/tokenizer_config.json',
 './gpt2_finetuned_model/special_tokens_map.json',
 './gpt2_finetuned_model/vocab.json',
 './gpt2_finetuned_model/merges.txt',
 './gpt2_finetuned_model/added_tokens.json')

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

model_path = "./gpt2_finetuned_model"

tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

tokenizer.pad_token = tokenizer.eos_token

def generate_text(prompt, max_length=100, temperature=0.9, top_k=40, top_p=0.85):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

    attention_mask = (input_ids != tokenizer.pad_token_id).long()

    output = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=max_length,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        repetition_penalty=1.2,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)

print(generate_text("The future of AI is",max_length=200))


The future of AI is uncertain . One way of thinking about AI is that it will not become a general @-@ purpose computer , although the next generation machine learning systems may eventually have similar capabilities . The idea was to make machines learn quickly enough to avoid mistakes or data leaks in which they might be more likely than to fail and to give them the ability ( at the cost of learning time ) when it comes down to errors in computation . But while such training might help with some of these problems , it would require a new level education for humans to be able do such training effectively . 
It is possible then, if we assume an increasing number of machines learning over longer periods of time , this could lead towards widespread adoption by AI practitioners as a tool for information processing . This might involve using " artificial intelligence — human @/@ machine learning — technology " , but I doubt that there can even possibly yet happen any such thing ; people oft

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!mv /content/lstm_model.keras "/content/drive/My Drive/lstm_model.keras"  # For TensorFlow/Keras

In [ ]:
!mv /content/tokenizer1.pkl "/content/drive/My Drive/tokenizer1.pkl"

In [ ]:
!mv /content/gpt2_finetuned/ "/content/drive/My Drive/gpt2_finetuned"

In [ ]:
!mv /content/gpt2_finetuned_model "/content/drive/My Drive/gpt2_finetuned_model"


#Flask API

In [ ]:
!pip install pyngrok

In [ ]:
pip install pyngrok

In [ ]:
!ngrok config add-authtoken "2eGRUzjZ2Ip4IGmRaR82OkqNxxN_4zfdjPoBPLwFn1EvSVa1v"

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
lstm_model = load_model("content/drive/MyDrive/lstm_model.keras", compile=False)

ValueError: File not found: filepath=content/drive/MyDrive/lstm_model.keras. Please ensure the file is an accessible `.keras` zip file.

In [ ]:
from flask import Flask, request, jsonify
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
from pyngrok import ngrok

app = Flask(__name__)

#LSTM
lstm_model = load_model("/content/drive/MyDrive/lstm_model.keras", compile=False)

with open("/content/drive/My Drive/tokenizer1.pkl", "rb") as f:
    tokenizer = pickle.load(f)

def generate_lstm_text(seed_text, next_words=50, max_sequence_length=10, temperature=0.8):
    """Generates text using the trained LSTM model with temperature scaling."""
    for _ in range(next_words):
        input_seq = tokenizer.texts_to_sequences([seed_text])
        input_seq = pad_sequences(input_seq, maxlen=max_sequence_length, padding="pre")

        predictions = lstm_model.predict(input_seq, verbose=0)[0]

        predictions = np.log(predictions + 1e-8) / temperature
        exp_preds = np.exp(predictions)
        probabilities = exp_preds / np.sum(exp_preds)

        predicted_word_index = np.random.choice(len(probabilities), p=probabilities)
        predicted_word = tokenizer.index_word.get(predicted_word_index, None)

        if predicted_word:
            seed_text += " " + predicted_word
        else:
            break

    return seed_text

#GPT2
model_path = "/content/drive/My Drive/gpt2_finetuned_model"
tokenizer_gpt = GPT2Tokenizer.from_pretrained(model_path)
gpt2_model = GPT2LMHeadModel.from_pretrained(model_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gpt2_model.to(device)

tokenizer_gpt.pad_token = tokenizer_gpt.eos_token

def generate_gpt_text(prompt, max_length=100, temperature=0.9, top_k=40, top_p=0.85):
    """Generates text using the GPT-2 model."""
    input_ids = tokenizer_gpt.encode(prompt, return_tensors="pt").to(device)

    attention_mask = (input_ids != tokenizer_gpt.pad_token_id).long()

    output = gpt2_model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=max_length,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        repetition_penalty=1.2,
        do_sample=True,
        pad_token_id=tokenizer_gpt.eos_token_id
    )

    return tokenizer_gpt.decode(output[0], skip_special_tokens=True)

#ROUTE
@app.route("/generate_lstm", methods=["POST"])
def generate_lstm():
    """Handles LSTM text generation requests."""
    try:
        data = request.get_json()
        seed_text = data.get("seed_text", "")
        next_words = int(data.get("next_words", 50))

        if not seed_text:
            return jsonify({"error": "Seed text is required"}), 400

        generated_text = generate_lstm_text(seed_text, next_words)
        return jsonify({"generated_text": generated_text})

    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route("/generate_gpt", methods=["POST"])
def generate_gpt():
    """Handles GPT-2 text generation requests."""
    try:
        data = request.get_json()
        prompt = data.get("prompt", "")
        max_length = int(data.get("max_length", 100))
        temperature = float(data.get("temperature", 0.9))
        top_k = int(data.get("top_k", 40))
        top_p = float(data.get("top_p", 0.85))

        if not prompt:
            return jsonify({"error": "Prompt is required"}), 400

        generated_text = generate_gpt_text(prompt, max_length, temperature, top_k, top_p)
        return jsonify({"generated_text": generated_text})

    except Exception as e:
        return jsonify({"error": str(e)}), 500



# Start Flask Server
port = 5000
public_url = ngrok.connect(port).public_url
print(f"🔥 Public API URL: {public_url}")

app.run(port=port)


🔥 Public API URL: https://519f-34-91-168-165.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
